# Abstract 

Digitization, as one of the key outcomes of technological growth, has led to profound changes in entertainment, and therefore in the world of cinema, as well as in many other areas. As a result, the distribution and broadcasting strategy that Netflix brought to the market turned into an amazing success story in a very short period of time.

Netflix's strategy is based on the idea that consumers can access the platform's entire content catalog for a monthly price. In addition, Netflix only broadcasts its films on the web, with no theatrical or limited distribution. The approach, which is vastly different from the classic idea of ​​the Hollywood studio system, has led to significant advances for audiences, directors and studios in various ways. In this way, we can confidently say that streaming services, such as Netflix, are influencing the film industry in terms of how we access movies, what material we consume and how movies are made.

Every day, platforms such as Netflix and Amazon Prime gain more users thanks to competitive prices compared to movie theaters, and recommendation algorithms. The latter play an important role in the dissemination of romantic comedies and thrillers, obtaining some success thanks to the data of millions of users who use them. This dominant position places Internet platforms in a strong position in terms of film content. In the future, that authority could be key in determining what constitutes a "well-made film".

The impact of Internet streaming services on filmmakers has been one of the most important transformations in the world of cinema in recent years. The promise of a more open environment for filmmakers than other large studios has attracted numerous directors to the platforms, with huge ramifications in the world of cinema. Furthermore, the fact that these services have less stringent standards than cinemas makes them attractive to producers. Another important aspect concerns independent directors. Since the 1980s, when Hollywood became the hub of cinema and blockbuster films began to dominate theaters, it has been difficult for independent directors to reach large audiences. Cinemas often prefer high-budget movies as they can make a much larger profit from them. As a result, independent films have few opportunities outside of film festivals to date. However, with internet streaming services becoming a major role in the world of cinema, independent filmmakers now have the opportunity to reach a wider audience.

The purpose of this notebook is to investigate, through data, how streaming platforms have changed film production. is the world of production really fairer? How much power does the user of these platforms have?

# Data gatering
We start from two existing datasets:
* [Netflix](https://www.kaggle.com/datasets/shivamb/netflix-shows): One of the most popular media and video streaming platforms. They have over 8000 movies or tv shows available on their platform, as of mid-2021, they have over 200M Subscribers globally. This tabular dataset consists of listings of all the movies and tv shows available on Netflix, along with details such as - cast, directors, ratings, release year, duration, etc.

* [Amazon prime](https://www.kaggle.com/datasets/shivamb/amazon-prime-movies-and-tv-shows): Another one of the most popular media and video streaming platforms. They have close to 10000 movies or tv shows available on their platform, as of mid-2021, they have over 200M Subscribers globally. This tabular dataset consists of listings of all the movies and tv shows available on Amazon Prime, along with details such as - cast, directors, ratings, release year, duration, etc.*



In [12]:
import pandas as pd # data processing
import pandas_profiling as pp
import numpy as np # linear algebra

In [13]:
df_netflix = pd.read_csv('originalDataset/netflix_titles.csv')
df_amazon = pd.read_csv('originalDataset/amazon_prime_titles.csv')

In [14]:
print(len(df_netflix))
print(len(df_amazon))

8807
9668


# Prepering data

Objective is the one of concatenate amazon and netflix databases mantaing storage information. W
We add two colums: netflix and amazon both with value 1 or 0 representing the absence or presence of the title on the platform. To keep the date added information we rename columns to distinguish the relative streaming service.

In [15]:
df_netflix.drop(columns = df_netflix.columns[0], axis = 1, inplace= True)
df_netflix['netflix'] = 1
df_netflix['amazon'] = 0
df_netflix.rename(columns = {'date_added':'date_added_netflix'}, inplace = True)

df_netflix.head(2)

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1,0
1,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",1,0


In [16]:
df_amazon.drop(columns = df_amazon.columns[0], axis = 1, inplace= True)
df_amazon['amazon'] = 1
df_amazon['netflix'] = 0
df_amazon.rename(columns = {'date_added':'date_added_amazon'}, inplace = True)

df_amazon.head(2)

,type,title,director,cast,country,date_added_amazon,release_year,rating,duration,listed_in,description,amazon,netflix
0,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,1,0
1,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,1,0


In [17]:

dataset = pd.concat([df_netflix, df_amazon],axis=0, join="outer", sort=False)

dataset.head(3)


,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1,0,NaN
1,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",1,0,NaN
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,NaN


The concatenated dataset present some errors. Shows that are both present on Netflix and Amazon Prime are recoded twice in our dataset: first time having only netflix information, the second time only the amazon ones. We decide to extract unic triples from the original datasets containing year, type (movie or tv Series) and name. Then we check the triples that the two dataset has in common and we store them in a list named 'title'.

In [18]:
netflix = []
amazon = []

def union(df, new):
    for  i, x in df['title'].iteritems():
        year = df['release_year'][i]
        type = df['type'][i]
        movie = x
        new.append((year, type, movie))
    return new

union(df_netflix, netflix)
union(df_amazon, amazon)

print(len(netflix), len(amazon))

8807 9668


In [19]:
title = []
for (y,t,m) in netflix:
    if (y,t,m) in amazon:
        title.append((y,t,m))

Finally, we iterate over the concatenated dataset querying only the shared titles merging amazon and Netflix information about possession and date of addition.
We decide to take description, country and cast information from the Netflix dataset because it was the best filled of the two. So, at the end of this process, we drop duplicates filtered by title, director, release year and type; keeping the first entries. Then we fill null value with 'No Data'.

In [20]:
df = dataset.copy()
df.replace(np.nan, 'null', inplace=True)
for i, r in df.iterrows(): 
    if (r['release_year'],r['type'],r['title']) in title:
        df.loc[i,'netflix'] = 1
        df.loc[i, 'amazon'] = 1
        q = df.query('title=="'+r['title']+'" & type=="'+r['type']+'" & release_year== '+ str(r['release_year']) +'')

        for i, x in q['date_added_netflix'].iteritems():
            if x != 'null':
                df.loc[i, 'date_added_netflix'] = x

        for i, x in q['date_added_amazon'].iteritems():
            if x != 'null':
                df.loc[i, 'date_added_amazon'] = x

In [21]:
df.replace( 'null', np.nan, inplace=True)
for i in df.columns:
    null_rate = df[i].isna().sum() / len(df) * 100 
    if null_rate > 0 :
        print("{} null rate: {}%".format(i,round(null_rate,2)))

director null rate: 25.53%
cast null rate: 11.14%
country null rate: 53.19%
date_added_netflix null rate: 51.4%
rating null rate: 1.85%
duration null rate: 0.02%
date_added_amazon null rate: 99.15%


In [22]:

df['date_added_netflix'].replace(np.nan, 1000,inplace  = True)
df['date_added_amazon'].replace(np.nan, 1000,inplace  = True)
df['country'].replace(np.nan, 'No Data',inplace  = True)
df['director'].replace(np.nan, 'No Data',inplace  = True)
df['cast'].replace(np.nan, 'No Data',inplace  = True)
df['rating'].replace(np.nan, 'No Data',inplace  = True)
df = df.drop_duplicates(subset=['title','director', 'release_year', 'type'], keep='first')
df = df.dropna()
df = df.reset_index(drop=True)

In [23]:
df['title'] = df['title'].replace({'"':''}, regex=True)
df['title'] = df['title'].replace({'\n':' '}, regex=True)

In [24]:
df

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Data,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1,0,1000
1,TV Show,Blood & Water,No Data,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",1,0,1000
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,1000
3,TV Show,Jailbirds New Orleans,No Data,No Data,No Data,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",1,0,1000
4,TV Show,Kota Factory,No Data,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,1,0,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18310,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",No Data,1000,1940,7+,60 min,Comedy,New York City street principles get an East Si...,0,1,1000
18311,TV Show,Planet Patrol,No Data,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",No Data,1000,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve...",0,1,1000
18312,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",No Data,1000,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou...",0,1,1000
18313,TV Show,Maradona: Blessed Dream,No Data,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",No Data,1000,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ...",0,1,1000


In [103]:
df.to_csv('data.csv')

In [9]:
df = pd.read_csv('data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Data,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1,0,1000
1,TV Show,Blood & Water,No Data,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",1,0,1000
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,1000
3,TV Show,Jailbirds New Orleans,No Data,No Data,No Data,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",1,0,1000
4,TV Show,Kota Factory,No Data,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,1,0,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18310,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",No Data,1000,1940,7+,60 min,Comedy,New York City street principles get an East Si...,0,1,1000
18311,TV Show,Planet Patrol,No Data,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",No Data,1000,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve...",0,1,1000
18312,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",No Data,1000,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou...",0,1,1000
18313,TV Show,Maradona: Blessed Dream,No Data,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",No Data,1000,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ...",0,1,1000


# Data enrichment 

Our data will be enriched using two sources:

* [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page): wikidata is the a free and open knowledge graph containing linked information used by the famouse online encyclopedia. 

* [iMdB](https://www.imdb.com/): Internet Movie Database is the world's most popular and authoritative source for movie, TV shows and celebrity content, where you can find ratings and reviews by creteque and public.

In [25]:
import pprint #indet json 
import requests #make http requests
from qwikidata.sparql  import return_sparql_query_results #return sparql results
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl
from http.client import IncompleteRead
import time
import urllib.error
from xml.etree.ElementPath import xpath_tokenizer_re

In order to query information, we split our dataset in Movies and TV Shows. Through wikidate we retrive missing information form our starting dataset, such like countries and directors. In addition, we add interesting information for us such as the gender of the director and the distributor. Finally, we also retrive the iMdb id of the movie for future query.

### Movies

In [41]:
movie_title = df.query("type == 'Movie'")
movie_title.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
movie_title.head(2)

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Data,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1,0,1000
1,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Data,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,1,0,1000


In [23]:
film = []
director = []
gender = []
distributor = []
imdbID = []
rottenscore = []
not_found = []

In [ ]:

def movie_director_gender(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query():
        
        try:
            my_SPARQL_query = """
            SELECT ?film_label ?director_label ?dir_gen_label 
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')
            OPTIONAL {?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')}}
            
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                not_found.append(""+x+"")
                
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                if "director_label" in results['results']['bindings'][0]:
                    director.append(results['results']['bindings'][0]['director_label']['value'])
                else:
                    director.append("no_data")
                if "dir_gen_label" in results['results']['bindings'][0]:
                    gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
                else:
                    gender.append("no_data")
                


        except urllib.error.HTTPError as e:
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            movie_director_gender(query[error_title:])
            

movie_director_gender(list(movie_title['title']))

In [ ]:

def movie_distributor_imdbid_rottenscore(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:

        try:

            
            my_SPARQL_query = """
            SELECT ?film_label ?distributor_label ?imdbID ?rottenscore
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            OPTIONAL {?film wdt:P345 ?imdbID.}
            OPTIONAL {?film wdt:P444 ?rottenscore.}
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                if "distributor_label" in results['results']['bindings'][0]:
                    distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                else:
                    distributor.append("no_data")
                if "imdbID" in results['results']['bindings'][0]:
                    imdbID.append(results['results']['bindings'][0]['imdbID']['value'])
                else:
                    imdbID.append("no_data")
                if "rottenscore" in results['results']['bindings'][0]:
                    rottenscore.append(results['results']['bindings'][0]['rottenscore']['value'])
                else:
                    rottenscore.append("no_data")



        except urllib.error.HTTPError as e:

            print(e.headers["retry-after"])
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            movie_distributor_imdbid_rottenscore(query[error_title:])
        except IncompleteRead:
            # Oh well, reconnect and keep trucking
            continue
            




movie_distributor_imdbid_rottenscore(list(movie_title['title']))

In [ ]:
movie_gender = {"title": film, "director": director,
        "director gender": gender}

movie_gender = pd.DataFrame(movie_gender)
movie_gender.to_csv('savelist.csv')
movie_gender.head(5)

In [ ]:
movie_info = {"title": film, "distributor": distributor, "id": imdbID, "rating score": rottenscore}

movie_info = pd.DataFrame(movie_info)
movie_info.to_csv('6000-9000.csv')
movie_info.head(5)

### TV Shows

In [34]:
tv_series = df.query("type == 'TV Show'")
tv_series.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
tv_series

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,TV Show,Blood & Water,No Data,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",1,0,1000
1,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,1000
2,TV Show,Jailbirds New Orleans,No Data,No Data,No Data,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",1,0,1000
3,TV Show,Kota Factory,No Data,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,1,0,1000
4,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,1,0,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4502,TV Show,Glass Room,No Data,"Elizaveta Maiskaya, Vsevolod Boldin, Alexander...",No Data,1000,2019,16+,1 Season,Drama,Geography teacher Elena lives with her husband...,0,1,1000
4503,TV Show,DaFUNdamentals Series,No Data,"Emile Crawford, Randall Crawford, Jon Ward",No Data,1000,2021,TV-Y,1 Season,"Animation, Kids","Learn ""daFundamentals"" through song and fun ch...",0,1,1000
4504,TV Show,Terrahawks,No Data,"DENISE BRYER, WINDSOR DAVIES, JEREMY HITCHEN, ...",No Data,1000,1983,ALL,3 Seasons,TV Shows,"This thrilling, action-packed series saw Gerry...",0,1,1000
4505,TV Show,Planet Patrol,No Data,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",No Data,1000,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve...",0,1,1000


In [35]:
len(tv_series)

4507

In [36]:
show = []
director = []
gender = []
distributor = []
imdbID = []
rottenscore = []
not_found = []

In [38]:

def series_director_gender(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:
        
        try:

            print(x)
            my_SPARQL_query = """
            SELECT ?series_label ?director_label ?dir_gen_label ?distributor_label ?imdbID ?rottenscore
            WHERE
            {
            ?series wdt:P31 wd:Q5398426 .
            ?series rdfs:label """+'"'+ x +'"' +"""@en .
            ?series rdfs:label ?series_label .
            FILTER(lang(?series_label) = 'en')
            OPTIONAL {?series wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')
            OPTIONAL {?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')}}
            OPTIONAL {?series wdt:P449 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            OPTIONAL {?series wdt:P345 ?imdbID.}
            OPTIONAL {?series wdt:P444 ?rottenscore.}
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                not_found.append(""+x+"")
                
            else:
                show.append(results['results']['bindings'][0]['series_label']['value'])
                if "director_label" in results['results']['bindings'][0]:
                    director.append(results['results']['bindings'][0]['director_label']['value'])
                else:
                    director.append("no_data")
                if "dir_gen_label" in results['results']['bindings'][0]:
                    gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
                else:
                    gender.append("no_data")
                if "distributor_label" in results['results']['bindings'][0]:
                    distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                else:
                    distributor.append("no_data")
                if "imdbID" in results['results']['bindings'][0]:
                    imdbID.append(results['results']['bindings'][0]['imdbID']['value'])
                else:
                    imdbID.append("no_data")
                if "rottenscore" in results['results']['bindings'][0]:
                    rottenscore.append(results['results']['bindings'][0]['rottenscore']['value'])
                else:
                    rottenscore.append("no_data")
                


        except urllib.error.HTTPError as e:
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            series_director_gender(query[error_title:])
            

series_director_gender(list(tv_series['title']))

Blood & Water
Ganglands
Jailbirds New Orleans
Kota Factory
Midnight Mass
The Great British Baking Show
Vendetta: Truth, Lies and The Mafia
Bangkok Breaking
Crime Stories: India Detectives
Dear White People
Falsa identidad
Jaguar
Monsters Inside: The 24 Faces of Billy Milligan
Resurrection: Ertugrul
Love on the Spectrum
Chicago Party Aunt
Sex Education
Squid Game
Tayo and Little Wizards
Angry Birds
Chhota Bheem
He-Man and the Masters of the Universe
The Smart Money Woman
Castle and Castle
Dharmakshetra
Nailed It
Numberblocks
Raja Rasoi Aur Anya Kahaniyan
Saved by the Bell
Stories by Rabindranath Tagore
Too Hot To Handle: Latino
Jack Whitehall: Travels with My Father
The World's Most Amazing Vacation Rentals
Yowamushi Pedal
Lucifer
Metal Shop Masters
Pokémon Master Journeys: The Series
Titipo Titipo
Mighty Raju
Into the Night
The Circle
The Circle
Kid Cosmic
Octonauts: Above & Beyond
On the Verge
Tobot Galaxy Detectives
Countdown: Inspiration4 Mission to Space
Tayo the Little Bus
Bunk'd


KeyboardInterrupt: 

In [ ]:

def series_distributor_imdbid_rottenscore(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:

        try:

            
            my_SPARQL_query = """
            SELECT ?series_label ?distributor_label ?imdbID ?rottenscore
            WHERE
            {
            ?series wdt:P31 wd:Q5398426 .
            ?series rdfs:label """+'"'+ x +'"' +"""@en .
            ?series rdfs:label ?series_label .
            FILTER(lang(?series_label) = 'en')
            OPTIONAL {?series wdt:P449 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            OPTIONAL {?series wdt:P345 ?imdbID.}
            OPTIONAL {?series wdt:P444 ?rottenscore.}
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                show.append(results['results']['bindings'][0]['film_label']['value'])
                if "distributor_label" in results['results']['bindings'][0]:
                    distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                else:
                    distributor.append("no_data")
                if "imdbID" in results['results']['bindings'][0]:
                    imdbID.append(results['results']['bindings'][0]['imdbID']['value'])
                else:
                    imdbID.append("no_data")
                if "rottenscore" in results['results']['bindings'][0]:
                    rottenscore.append(results['results']['bindings'][0]['rottenscore']['value'])
                else:
                    rottenscore.append("no_data")



        except urllib.error.HTTPError as e:

            print(e.headers["retry-after"])
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            series_distributor_imdbid_rottenscore(query[error_title:])
        except IncompleteRead:
            # Oh well, reconnect and keep trucking
            continue
            




series_distributor_imdbid_rottenscore(list(tv_series['title']))

In [42]:
serie = {"id": imdbID, "title": show, "director": director, "director_gender": gender, "distributor": distributor, "rating score": rottenscore}
serie = pd.DataFrame(serie)
serie = serie.drop_duplicates()
serie.to_csv('TVShows.csv')
serie

,id,title,director,director_gender,distributor,rating score
0,no_data,Blood & Water,no_data,no_data,no_data,no_data
1,no_data,Ganglands,no_data,no_data,no_data,no_data
2,tt9839146,Blood & Water,no_data,no_data,Netflix,no_data
4,tt10574558,Midnight Mass,Mike Flanagan,male,Netflix,no_data
5,tt1877368,The Great British Baking Show,no_data,no_data,BBC One,no_data
...,...,...,...,...,...,...
1990,tt3973820,Red Oaks,no_data,no_data,Amazon Prime Video,no_data
1991,tt14344724,The Flaming Heart,no_data,no_data,no_data,no_data
1992,no_data,Saiyaan Way,no_data,no_data,no_data,no_data
1993,tt0085099,Terrahawks,Desmond Saunders,male,ITV,no_data


In [ ]:
serie_info = {"title": show, "distributor": distributor, "id": imdbID, "rating score": rottenscore}

serie_info = pd.DataFrame(serie_info)
serie_info.to_csv('series.csv')
serie_info.head(5)

# data analysis

# data visualization

In [ ]:
#